In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import lightgbm as lgb
from tqdm.auto import tqdm

In [2]:
DATA_ROOT   = Path('./data')
WORKING_DIR = Path('./')
STUDY_PATH = WORKING_DIR / 'studies'

In [3]:
n_splits = 5
cv = KFold(n_splits=n_splits, shuffle=False)

X = pd.read_parquet(DATA_ROOT / 'working_dataset.parquet')
y = pd.read_csv(DATA_ROOT / 'train_labels.csv')['imdb_score']

In [4]:
class VotingModel(BaseEstimator, RegressorMixin):
    def __init__(self, estimators):
        super().__init__()
        self.estimators = estimators
        
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        y_preds = [estimator.predict(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)
    
    def predict_proba(self, X):
        try:
            y_preds = [estimator.predict_proba(X) for estimator in self.estimators]
        except AttributeError:
            y_preds = [estimator.predict(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)

In [5]:
def train(params):
    fitted_models = []
    cv_scores = []


    for idx_train, idx_valid in tqdm(cv.split(X, y), total=n_splits):
        X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
        X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

        model = lgb.LGBMRegressor(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            callbacks=[lgb.log_evaluation(200), lgb.early_stopping(60)]
            # callbacks=[lgb.early_stopping(50, verbose=False)]
        )
        fitted_models.append(model)
    
        y_pred = model.predict(X_valid)

        mse_score = mean_squared_error(y_valid, y_pred)
        cv_scores.append(mse_score)
    
    model = VotingModel(fitted_models)

    return model, cv_scores

In [6]:
import joblib

# Create or load the study
def create_or_load_study(study_name, storage=None):
    try:
        study = joblib.load(f"{STUDY_PATH}/{study_name}.pkl")
        print(f"Loaded study '{study_name}' from file.")
    except FileNotFoundError:
        study = optuna.create_study(study_name=study_name, storage=storage, direction='maximize')
        print(f"Created new study '{study_name}'.")
    return study

# Function to save the study
def save_study(study, study_name):
    joblib.dump(study, f"{STUDY_PATH}/{study_name}.pkl")
    print(f"Study '{study_name}' saved to file.")

In [ ]:
import optuna

def objective(trial):
    params = {
            'metric': 'rmse', 
            'random_state': 48,
            'n_estimators': 20000,
            'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
            'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
            'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
            'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
            'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
            'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
            'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
            'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100),
            'verbose': -1
        }

    model, cv_scores = train(params)

    #print(f'======= Mean of scores: {np.mean(cv_scores):.3f}±{2*np.std(cv_scores):.3f} =======')
    return np.mean(cv_scores)


study_name = 'optuna'
study = create_or_load_study(study_name)

def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    #print(f"Best value: {study.best_value}, Best params: {trial.best_trial.params}")

    study_df = study.trials_dataframe()
    save_study(study, study_name)
    study_df.to_csv('optuna.csv', index=False)


study.optimize(objective, n_trials=40, callbacks=[print_callback], n_jobs=1, show_progress_bar=True)

[I 2024-04-05 20:55:16,582] A new study created in memory with name: optuna


Created new study 'optuna'.


  0%|          | 0/40 [00:00<?, ?it/s]

/tmp/ipykernel_10335/418579513.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
/tmp/ipykernel_10335/418579513.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),


  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

Training until validation scores don't improve for 60 rounds
[200]	valid_0's rmse: 0.469946
[400]	valid_0's rmse: 0.449291
[600]	valid_0's rmse: 0.442998
[800]	valid_0's rmse: 0.439146
[1000]	valid_0's rmse: 0.437508
[1200]	valid_0's rmse: 0.435623
[1400]	valid_0's rmse: 0.434506
[1600]	valid_0's rmse: 0.433242
[1800]	valid_0's rmse: 0.432141
[2000]	valid_0's rmse: 0.431279
Early stopping, best iteration is:
[2031]	valid_0's rmse: 0.431162
Training until validation scores don't improve for 60 rounds
[200]	valid_0's rmse: 0.453932
[400]	valid_0's rmse: 0.434012
[600]	valid_0's rmse: 0.429318
[800]	valid_0's rmse: 0.426729
[1000]	valid_0's rmse: 0.425541
Early stopping, best iteration is:
[1072]	valid_0's rmse: 0.425171
Training until validation scores don't improve for 60 rounds
[200]	valid_0's rmse: 0.495552
[400]	valid_0's rmse: 0.472153
[600]	valid_0's rmse: 0.466904
[800]	valid_0's rmse: 0.464745
[1000]	valid_0's rmse: 0.462774
[1200]	valid_0's rmse: 0.461387
[1400]	valid_0's rmse: 

/tmp/ipykernel_10335/418579513.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
/tmp/ipykernel_10335/418579513.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),


  0%|          | 0/5 [00:00<?, ?it/s]

Training until validation scores don't improve for 60 rounds
[200]	valid_0's rmse: 0.499189
[400]	valid_0's rmse: 0.424674
[600]	valid_0's rmse: 0.408244
[800]	valid_0's rmse: 0.402268
[1000]	valid_0's rmse: 0.399629
[1200]	valid_0's rmse: 0.398332
[1400]	valid_0's rmse: 0.397217
[1600]	valid_0's rmse: 0.395715
[1800]	valid_0's rmse: 0.395278
Early stopping, best iteration is:
[1761]	valid_0's rmse: 0.395188
Training until validation scores don't improve for 60 rounds
[200]	valid_0's rmse: 0.4719
[400]	valid_0's rmse: 0.394072
[600]	valid_0's rmse: 0.385059
Early stopping, best iteration is:
[649]	valid_0's rmse: 0.384579
Training until validation scores don't improve for 60 rounds
[200]	valid_0's rmse: 0.530673
[400]	valid_0's rmse: 0.452309
[600]	valid_0's rmse: 0.432603
[800]	valid_0's rmse: 0.425043
[1000]	valid_0's rmse: 0.42053
[1200]	valid_0's rmse: 0.418256
[1400]	valid_0's rmse: 0.416873
[1600]	valid_0's rmse: 0.41582
Early stopping, best iteration is:
[1619]	valid_0's rmse: 0

In [ ]:
params = {
        'metric': 'rmse', 
        'random_state': 48,
        'n_estimators': 20000,
        #'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        #'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        #'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        #'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        #'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        #'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        #'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        #'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        #'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100),
        'verbose': -1

    }
    
model, cv_scores = train(params)

In [ ]:
f'{np.mean(cv_scores): .4f} ± {2*np.std(cv_scores): .4f}' # before 0.8577 ± 0.0048

## Feature Importance

In [ ]:
import shap

shap_acc = None
X_acc = None

for i, (idx_train, idx_valid) in tqdm(enumerate(cv.split(X, y)), total=n_splits):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
    explainer = shap.TreeExplainer(model.estimators[i])
    shap_values = explainer.shap_values(X_train)
    
    if shap_acc is not None:
        shap_acc = np.concatenate([shap_acc, shap_values])
    else:
        shap_acc = shap_values
    
    if X_acc is not None:
        X_acc = pd.concat([X_acc, X_train])
    else:
        X_acc = X_train

In [ ]:
X.shape

In [ ]:
shap.summary_plot(shap_acc, X_acc)